### YOLOv5 installation
NB! Meant to be run in Google Colab.

### As long as you change the *path_to_project* variable to your path up until the project and *project_root* NATO-Symbols part to the project folder name, code should run seamlessly.

In [1]:
import os

path_to_project="/content/drive/MyDrive/Yolov5"
project_root=path_to_project+"/NATO-Symbols"
yolo_dir=project_root+"/yolov5"
#NOTE THAT YOLO EXPECTS THE datasets FOLDER TO BE IN THE SAME ROOT FOLDER AS IT IS
img_dir=project_root+"/datasets/NATO-Symbols/images"
label_dir=project_root+"/datasets/NATO-Symbols/labels"
if yolo_dir==None:
  yolo_dir=input("Please paste the directory you want to install YOLOv5 into. WITHOUT / IN THE END")

%cd {project_root}

![ -d {yolo_dir} ] && echo "YOLO DIRECTORY EXISTS SKIPPING INSTALL"
![ ! -d {yolo_dir} ] && echo "INSTALLING YOLOv5 INTO DESIGNATED DIRECTORY" && git clone https://github.com/ultralytics/yolov5


/content/drive/MyDrive/Yolov5/NATO-Symbols
YOLO DIRECTORY EXISTS SKIPPING INSTALL


### Requirements installation
NB! Meant to be run in Google Colab.

In [2]:
import sys
sys.path.append(project_root+"/scripts")
yolo_reqs=yolo_dir+"/requirements.txt"
project_reqs=project_root+"/requirements.txt"
!pip install wandb
!pip install -r  {yolo_reqs}
!pip install -r {project_reqs}

     |████████████████████████████████| 1.8 MB 27.7 MB/s 
     |████████████████████████████████| 145 kB 67.2 MB/s 
     |████████████████████████████████| 181 kB 70.3 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=3e2aa58706a43c7ebbb20d69d1ac6ec8f2a5ede5b03e49e3eeb5ca6f2e989a11
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
     |████████████████████████████████| 596 kB 27.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Generate a YOLOv5 dataset

This is based on single-symbol images that will be randomly placed on a white canvas to compose a set of multi-symbol images that will be used in training the YOLOv5 weights.

In [3]:
import numpy as np
import cv2
from random import randint
from scipy import ndimage
import os
import re
from tqdm import tqdm
from augment_tools import augment
from matplotlib import pylab as plt

dataset_sz=200
dim=(850,850)
label_list=["advance_to_contact", "attack", "block", "counterattack", "cover", "delay", "destroy", "disrupt", "guard", "occupy", "retain", "screen", "secure", "seize", "support_by_fire"]

In [4]:
raws=[]
raw_labels=[]
raw_dir=project_root+"/datasets/raw/"
file_list=os.listdir(raw_dir)

for title in file_list:
  raw=cv2.imread(raw_dir+title,0)

  raw=raw[np.argwhere(np.amin(raw,axis=1)<120)[0][0]:np.argwhere(np.amin(raw,axis=1)<120)[-1][0],:]
  raw = raw[:,np.argwhere(np.amin(raw,axis=0) < 120)[0][0]:np.argwhere(np.amin(raw,axis=0) < 120)[-1][0]]

  raws.append(raw)
  raw_labels.append(re.findall('([a-zA-Z_ ]*)\d*.*', title)[0])

In [30]:
def generate_image(raw, raw_labels, dim=(850,850),min_sym=2,max_sym=6, max_overlap=15):
  canvas=np.full(dim,255)
  sym_data=[]
  locations=[]

  for i in range(randint(min_sym,max_sym)):
    choice_idx=randint(0,len(raw)-1)
    chosen=np.copy(raw[choice_idx])
    #chosen=augment(chosen, apply_resize=False, apply_flip=True, apply_rotation=True, apply_transformation=True, apply_boldness=True)

    while(True):
      overlap=False
      top_leftX=randint(0,dim[0]-chosen.shape[0])
      top_leftY=randint(0,dim[1]-chosen.shape[1])
      for j in range(len(locations)):
        if ((top_leftX < locations[j][1][0]-max_overlap) and (top_leftY < locations[j][1][1]-max_overlap) and (top_leftX+chosen.shape[0] > locations[j][0][0]+max_overlap) and (top_leftY+chosen.shape[1] > locations[j][0][1]+max_overlap)):
          overlap=True
          break
      if not overlap:
        break

    locations.append([[top_leftX,top_leftY],[top_leftX+chosen.shape[0],top_leftY+chosen.shape[1]]])
    canvas[top_leftX:(top_leftX+chosen.shape[0]),top_leftY:(top_leftY+chosen.shape[1])][chosen < 140] = chosen[chosen < 140]
    
    #YOLO xywh format, flip x and y and w and h from previous step
    dimX, dimY = dim
    y=round(((top_leftX+(chosen.shape[0]/2))/dimX),2)
    x=round(((top_leftY+(chosen.shape[1]/2))/dimY),2)
    h=round((chosen.shape[0]/dimX),2)
    w=round((chosen.shape[1]/dimY),2)

    #Class nr
    label=raw_labels[choice_idx]
    class_nr=label_list.index(label)
    sym_data.append([class_nr, x, y, w, h])


  return canvas, sym_data

In [31]:
if not os.path.exists(img_dir+"/train/img0.jpg"):
  canvases=np.zeros((dataset_sz,dim[0],dim[1]))
  canvases_data=[]

  for i in tqdm(range(dataset_sz)):
    canvases[i], canvas_data = generate_image(raws, raw_labels, dim)
    canvases_data.append(canvas_data)

  trainnr=int(0.8*len(canvases))

  #This is a lot of for loops but the written data sizes are small so it is ok for this use-case
  for i in range(len(canvases)):
    if i<trainnr:
      if cv2.imwrite(img_dir+"/train/img"+str(i)+".jpg", canvases[i]):
        with open(label_dir+"/train/img"+str(i)+".txt", 'w') as f:
          for j in canvases_data[i]:
            for k in j:
              f.write(str(k)+" ")
            f.write("\n")
    else:
      if cv2.imwrite(img_dir+"/val/img"+str(i)+".jpg", canvases[i]):
        with open(label_dir+"/val/img"+str(i)+".txt", 'w') as f:
          for j in canvases_data[i]:
            for k in j:
              f.write(str(k)+" ")
            f.write("\n")


100%|██████████| 200/200 [00:01<00:00, 129.89it/s]


### Training YOLOv5

In [8]:
import wandb

In [9]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [32]:
!python3 /content/drive/MyDrive/Yolov5/NATO-Symbols/yolov5/train.py --img 480 --batch 64 --epochs 100 --data NATO-Symbols.yaml --weights yolov5m.pt --project NATO-Symbols-Log --name 5m-100

wandb: Currently logged in as: erkoiv. Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, cfg=, data=NATO-Symbols.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=64, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=NATO-Symbols-Log, name=5m-100, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-211-gcee5959 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1